In [4]:
from astropy.table import Table
import numpy as np
import matplotlib.pyplot as plt
import glob
import os
import fitsio
import desimodel.io
import desitarget.mtl
import desisim.quickcat
from astropy.io import fits
from astropy.table import Table, Column, vstack
import json
import shutil
import healpy
import mtl as mymtl
import targets


from desitarget.targetmask import desi_mask, obsconditions
from collections import Counter
import subprocess

%matplotlib inline

# Global statistics

In [5]:
def consolidate_favail(fba_files):
    # getting all the targetids of the assigned fibers
    print('reading individual fiberassign files')
    favail = list()
    for i_tile, tile_file in enumerate(fba_files):
        if i_tile%50 ==0:
            print(i_tile)
        id_favail, header = fits.getdata(tile_file, 'FAVAIL', header=True)
        favail.extend(id_favail['TARGETID'])
    return list(set(favail))

def global_efficiency(targets, id_avail, zcat, target_class='QSO', zcat_spectype='QSO', z_max=None, z_min=None):
    ii_avail = np.in1d(targets['TARGETID'], id_avail)
    targets_avail = targets[ii_avail]

    if z_max is None and z_min is None:
        sub_zcat = zcat.copy()
    elif (z_min is not None) or (z_max is not None):
        if z_max is not None:
            sub_zcat = zcat[zcat['Z']<z_max]
        if z_min is not None:
            sub_zcat = zcat[zcat['Z']>z_min]
    else:
        print("Error")
        sub_zcat = None

    # input target consistent with target_class
    is_class = (targets_avail['DESI_TARGET'] & desi_mask.mask(target_class))!=0
    targets_avail_class = targets_avail[is_class]
    n_avail = len(targets_avail_class)

    # output in the redshift catalog consistent with truth_spectype
    sub_zcat_class = sub_zcat[sub_zcat['SPECTYPE']==zcat_spectype]
    
    # keep the elements in the zcat that correspond to the correct input target class
    id_intersection = np.in1d(sub_zcat_class['TARGETID'], targets_avail_class['TARGETID'])
    sub_zcat_class = sub_zcat_class[id_intersection]
    n_assigned = len(sub_zcat_class)

    nobs = dict()
    for i in range(10):
        nobs[i] = np.count_nonzero(sub_zcat_class['NUMOBS']==i)
    nobs[0] = (n_avail - n_assigned)

    print(target_class, zcat_spectype, n_assigned/n_avail, n_avail, n_assigned, nobs)
    
def tile_efficiency(qa_json_file):
    f = open(qa_json_file)
    qa_dict = json.load(f)
    f.close()
    assign_total = []
    assign_science= []
    assign_sky = []
    assign_std = []
    for k in qa_dict:
        assign_total.append(qa_dict[k]['assign_total'])
        assign_science.append(qa_dict[k]['assign_science'])
        assign_sky.append(qa_dict[k]['assign_sky'])
        assign_std.append(qa_dict[k]['assign_std'])
    assign_total = np.array(assign_total)
    assign_science = np.array(assign_science)
    assign_sky = np.array(assign_sky)
    assign_std = np.array(assign_std)
    n_not_enough_sky = np.count_nonzero(assign_sky<400)
    n_not_enough_std = np.count_nonzero(assign_std<100)
    f_unassigned = (5000 - assign_total)/5000
    print(n_not_enough_sky, n_not_enough_std, np.median(f_unassigned))

In [3]:
! mkdir strategy_A/fiberassign_full
! cp strategy_A/fiberassign_dark0/tile*.fits strategy_A/fiberassign_full/
! cp strategy_A/fiberassign_dark1/tile*.fits strategy_A/fiberassign_full/
! cp strategy_A/fiberassign_dark2_dark3/tile*.fits strategy_A/fiberassign_full/
! cp strategy_A/fiberassign_gray/tile*.fits strategy_A/fiberassign_full/
! fba_run_qa --dir strategy_A/fiberassign_full/

mkdir: cannot create directory ‘strategy_A/fiberassign_full’: File exists
INFO: Loaded focalplane for time stamp 2019-09-24 21:22:03.316953
INFO: Focalplane has 0 fibers that are stuck / broken
INFO: Found 245 fiberassign tile files
INFO: Selecting 245 fiberassign tile files
INFO: Focalplane has 0 fibers that are stuck / broken
INFO: Focalplane has 0 fibers that are stuck / broken
INFO: Focalplane has 0 fibers that are stuck / broken
INFO: Focalplane has 0 fibers that are stuck / broken
INFO: Focalplane has 0 fibers that are stuck / broken
INFO: Processing tile 22715
INFO: Focalplane has 0 fibers that are stuck / broken
INFO: Processing tile 28523
INFO: Processing tile 22743
INFO: Focalplane has 0 fibers that are stuck / broken
INFO: Processing tile 5469
INFO: Processing tile 11227
INFO: Focalplane has 0 fibers that are stuck / broken
INFO: Processing tile 16970
INFO: Processing tile 28485
INFO: Focalplane has 0 fibers that are stuck / broken
INFO: Processing tile 5431
INFO: Processing

In [6]:
fba_files_A = glob.glob('strategy_A/fiberassign_full/tile-*fits')
favail_A = consolidate_favail(fba_files_A)
zcat_A = Table.read('strategy_A/zcat/dark2_dark3_zcat.fits')
tile_efficiency('strategy_A/fiberassign_full/qa.json')
targets = Table.read("targets/subset_dr8_mtl_dark_gray_NGC.fits")
truth = Table.read("targets/subset_truth_dr8_mtl_dark_gray_NGC.fits")
targets.sort(keys='TARGETID')
zcat_A.sort(keys='TARGETID')
truth.sort(keys='TARGETID')


reading individual fiberassign files
0
50
100
150
200
0 0 0.0


In [6]:
global_efficiency(targets, favail_A, zcat_A, target_class='QSO', zcat_spectype='QSO')
global_efficiency(targets, favail_A, zcat_A, target_class='LRG', zcat_spectype='GALAXY')
global_efficiency(targets, favail_A, zcat_A, target_class='ELG', zcat_spectype='GALAXY')
global_efficiency(targets, favail_A, zcat_A, target_class='QSO', zcat_spectype='QSO', z_min=2.15)
global_efficiency(targets, favail_A, zcat_A, target_class='QSO', zcat_spectype='QSO', z_max=2.15)
#global_efficiency(targets, favail_A, zcat_A, target_class='LRG_1PASS', zcat_spectype='GALAXY')
#global_efficiency(targets, favail_A, zcat_A, target_class='LRG_2PASS', zcat_spectype='GALAXY')

QSO QSO 0.9742884108209185 113373 110458 {0: 2915, 1: 69728, 2: 12540, 3: 12411, 4: 15773, 5: 4, 6: 2, 7: 0, 8: 0, 9: 0}
LRG GALAXY 0.8986254647506479 195266 175471 {0: 19795, 1: 134464, 2: 41007, 3: 0, 4: 0, 5: 0, 6: 0, 7: 0, 8: 0, 9: 0}
ELG GALAXY 0.592170350830815 1051618 622737 {0: 428881, 1: 622736, 2: 1, 3: 0, 4: 0, 5: 0, 6: 0, 7: 0, 8: 0, 9: 0}
QSO QSO 0.16916726204651902 113373 19179 {0: 94194, 1: 1654, 2: 2068, 3: 4510, 4: 10945, 5: 1, 6: 1, 7: 0, 8: 0, 9: 0}
QSO QSO 0.8051211487743996 113373 91279 {0: 22094, 1: 68074, 2: 10472, 3: 7901, 4: 4828, 5: 3, 6: 1, 7: 0, 8: 0, 9: 0}


In [11]:
! mkdir strategy_B/fiberassign_full
! cp strategy_B/fiberassign_dark0/tile*.fits strategy_B/fiberassign_full/
! cp strategy_B/fiberassign_dark1/tile*.fits strategy_B/fiberassign_full/
! cp strategy_B/fiberassign_dark2_dark3/tile*.fits strategy_B/fiberassign_full/
! cp strategy_B/fiberassign_gray/tile*.fits strategy_B/fiberassign_full/
! fba_run_qa --dir strategy_B/fiberassign_full/

mkdir: cannot create directory ‘strategy_B/fiberassign_full’: File exists
INFO: Loaded focalplane for time stamp 2019-09-25 14:59:51.887444
INFO: Focalplane has 0 fibers that are stuck / broken
INFO: Found 0 fiberassign tile files
INFO: Selecting 0 fiberassign tile files


In [12]:
fba_files_B = glob.glob('strategy_B/fiberassign_full/tile-*fits')
favail_B = consolidate_favail(fba_files_B)
zcat_B = Table.read('strategy_B/zcat/dark2_dark3_zcat.fits')
tile_efficiency('strategy_B/fiberassign_full/qa.json')
targets.sort(keys='TARGETID')
zcat_B.sort(keys='TARGETID')
truth.sort(keys='TARGETID')

reading individual fiberassign files
0
50
100
150
200


/global/common/software/desi/cori/desiconda/20190804-1.3.0-spec/conda/lib/python3.6/site-packages/numpy/core/fromnumeric.py:3118: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/global/common/software/desi/cori/desiconda/20190804-1.3.0-spec/conda/lib/python3.6/site-packages/numpy/core/_methods.py:85: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


0 0 nan


In [13]:
global_efficiency(targets, favail_B, zcat_B, target_class='QSO', zcat_spectype='QSO')
global_efficiency(targets, favail_B, zcat_B, target_class='LRG', zcat_spectype='GALAXY')
global_efficiency(targets, favail_B, zcat_B, target_class='ELG', zcat_spectype='GALAXY')
global_efficiency(targets, favail_B, zcat_B, target_class='QSO', zcat_spectype='QSO', z_min=2.15)
global_efficiency(targets, favail_B, zcat_B, target_class='QSO', zcat_spectype='QSO', z_max=2.15)

QSO QSO 0.9742972312631756 113373 110459 {0: 2914, 1: 69806, 2: 12428, 3: 12354, 4: 15864, 5: 5, 6: 2, 7: 0, 8: 0, 9: 0}
LRG GALAXY 0.8985332828039699 195266 175453 {0: 19813, 1: 134460, 2: 40993, 3: 0, 4: 0, 5: 0, 6: 0, 7: 0, 8: 0, 9: 0}
ELG GALAXY 0.600006846592584 1051618 630978 {0: 420640, 1: 630977, 2: 1, 3: 0, 4: 0, 5: 0, 6: 0, 7: 0, 8: 0, 9: 0}
QSO QSO 0.16916726204651902 113373 19179 {0: 94194, 1: 1654, 2: 2064, 3: 4500, 4: 10959, 5: 1, 6: 1, 7: 0, 8: 0, 9: 0}
QSO QSO 0.8051299692166565 113373 91280 {0: 22093, 1: 68152, 2: 10364, 3: 7854, 4: 4905, 5: 4, 6: 1, 7: 0, 8: 0, 9: 0}


In [14]:
110459+175453+630978

916890

In [15]:
110458+175471+622737

908666

In [ ]:
truth[:10]

In [ ]:
avail_a_array = np.array([favail_A])
avail = Table(np.transpose(avail_a_array), names=(['TARGETID']))
avail.write("available.fits", overwrite=True)
avail = Table.read("available.fits")

In [ ]:
compiled = targets.copy()
ii = np.in1d(targets['TARGETID'], avail['TARGETID'])
compiled = targets[ii]

In [ ]:
compiled['NUMOBS'] = np.zeros(len(compiled), dtype=int)
compiled['Z'] = np.zeros(len(compiled))
compiled['TRUEZ'] = np.zeros(len(compiled), dtype=int)
compiled['TRUESPECTYPE'] = np.repeat('A', len(compiled))


ii_from_z = np.in1d(compiled['TARGETID'], zcat_A['TARGETID'])
compiled['Z'][ii_from_z] = zcat_A['Z']
compiled['NUMOBS'][ii_from_z] = np.int_(zcat_A['NUMOBS'])

ii_from_truth = np.in1d(truth['TARGETID'], compiled['TARGETID'])
compiled['TRUESPECTYPE'] = truth['TRUESPECTYPE'][ii_from_truth]
compiled['TRUEZ'] = truth['TRUEZ'][ii_from_truth]

In [ ]:
compiled[['RA', 'DEC', 'TARGETID', 'DESI_TARGET', 'NUMOBS', 'TRUESPECTYPE', 'TRUEZ']].write("myzcat_strategy_A.fits", overwrite=True)

In [ ]:
compiled=Table.read("myzcat_strategy_A.fits")

In [ ]:
rr = np.random.random(len(compiled))
ii = rr<0.01
plt.scatter(compiled['RA'][ii], compiled['DEC'][ii], s=0.1)

In [ ]:
compiled